In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

def download_data(dd):
    
    import os
    import glob
    import datetime
    import numpy as np
    import pandas as pd
    import time
    import matplotlib.pyplot as plt
    %matplotlib inline
    import pymongo
    import io
    import pandas as pd
    import pickle
    import datetime
    import time
    import gzip
    import lzma
    import pytz
    import pyarrow as pa
    import pyarrow.parquet as pq
    import numpy as np

    print('start processing raw Data')
    startTm = datetime.datetime.now()

    startDate = dd
    endDate = dd

    readPath = '/mnt/equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

    date = dateLs[0]
    assert(date == dd)

    readPath = '/mnt/equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%date))

    for col in ['clockAtArrival', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        orderLog[col] = orderLog[col].fillna(0)
        orderLog[col] = orderLog[col].astype('int64')

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    orderLog = orderLog[orderLog["secid"] >= 1000000]

    targetStock = orderLog['secid'].unique()
    targetStock = np.array([int(str(i)[1:]) for i in targetStock])
    targetStockSZ = sorted(targetStock[targetStock < 600000])
    targetStockSH = sorted(targetStock[targetStock >= 600000])

    readPath = '/mnt/Kevin_zhenyu/rawData'
    mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
    mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]

    mdOrderLog = pd.read_csv(mdOrderLogPath)
    mdOrderLog = mdOrderLog[mdOrderLog['SecurityID'].isin(targetStockSZ)]
    mdOrderLog['OrderType'] = mdOrderLog['OrderType'].astype(str)
    mdOrderLog = mdOrderLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                         'OrderType', 'Price', 'OrderQty']]

    mdTradeLog = pd.read_csv(mdTradeLogPath, encoding='utf-8')
    mdTradeLog['ExecType'] = mdTradeLog['ExecType'].astype(str)
    mdTradeLog = mdTradeLog[mdTradeLog['SecurityID'].isin(targetStockSZ)]
    mdTradeLog['volumeThisUpdate'] = np.where(mdTradeLog['ExecType'] == 'F', mdTradeLog['TradeQty'], 0)
    mdTradeLog['cum_volume'] = mdTradeLog.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
    mdTradeLog = mdTradeLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                         'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]

    rawMsgDataSZ = pd.concat([mdOrderLog, mdTradeLog], sort=False)
    del mdOrderLog
    del mdTradeLog

    rawMsgDataSZ = rawMsgDataSZ.sort_values(by=['sequenceNo']).reset_index(drop=True)

    rawMsgDataSZ['cum_volume'] = rawMsgDataSZ.groupby(['SecurityID'])['cum_volume'].ffill().bfill()
    rawMsgDataSZ['ExecType'] = rawMsgDataSZ['ExecType'].fillna('2')
    rawMsgDataSZ['TradeQty'] = rawMsgDataSZ['TradeQty'].fillna(0)

    saveCols = ['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'cum_volume', 'ApplSeqNum', 
            'Side', 'OrderType', 'Price', 'OrderQty', 'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney',
            'BidApplSeqNum', 'OfferApplSeqNum']
    rawMsgDataSZ = rawMsgDataSZ[saveCols]
    savePath = '/mnt/orderLog/mdData'
    rawMsgDataSZ.to_pickle(os.path.join(savePath, 'mdLog_msg_%s.pkl'%startDate))
    print(datetime.datetime.now() - startTm)






    print('start tickToMBD')

    pd.set_option('max_rows', 200)
    pd.set_option('max_columns', 200)

    perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

    startTm = datetime.datetime.now()

    startDate = dd
    endDate = dd

    readPath = '/mnt/equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

    thisDate = dateLs[0]
    assert(thisDate == dd)

    readPath = '/mnt/equityTradeLogs'
    rawOrderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
    for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        rawOrderLog[col] = rawOrderLog[col].fillna(0)
        rawOrderLog[col] = rawOrderLog[col].astype('int64')   
    rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    original_data = rawOrderLog.copy()

    rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)]\
    .groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep='first')

    display('There are ticks with orderDirection 0')
    display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
                'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

    assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    try:
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    except:
        print('There are orders with all things same except sdd')
        print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)])
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
    try:
        assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
    except:
        print('There are orders with same internalId but different orderId other than accCode 8856 case')
        print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

    r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize']).grouper.group_info[0]
    r1 = r1.sort_values(by=['test', 'clockAtArrival'])
    r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
    r1['vai'] = r1.groupby('test')['vai'].ffill()
    r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    r2 = r2.sort_values(by=['test', 'clockAtArrival'])
    r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
    r2['vai'] = r2.groupby('test')['vai'].ffill()
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    try:
        assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    rawOrderLog = pd.concat([r1, r2])
    del r1
    del r2  

    rawOrderLog = rawOrderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
    rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog['accCode'] // 100)
    rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
    rawOrderLog['colo_account'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['accCode'].astype('str')
    rawOrderLog['order'] = rawOrderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize']).grouper.group_info[0]
    rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['clockAtArrival'].transform('first')
    rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
    rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
    rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
    rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
        rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
    orderLog = rawOrderLog.copy()
    del rawOrderLog
    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

    orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')
    orderLog = orderLog[orderLog['firstUpdateType'] == 0]
    orderLog['insertNum'] = np.where(orderLog['updateType'] == 0, 1, 0)
    orderLog['insertNum'] = orderLog.groupby(['order'])['insertNum'].transform('sum')
    orderLog = orderLog[orderLog['insertNum'] == 1]
    orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)
    def getTuple(x):
        return tuple(i for i in x)

    # 1. market orders
    orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    orderDataSZ['updateType'] = 0
    tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                             rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
    tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
    tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
    tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
    tradeDataSZ['updateType'] = 4
    cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
    cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
    cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
    cancelDataSZ['updateType'] = 3

    msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
    del orderDataSZ
    del tradeDataSZ
    del cancelDataSZ
    msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
    msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
    msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
    msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
    msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
    msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
    assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
    msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
    msgDataSZ['date'] = int(thisDate) 
    print('finish market orders')


    # 2. orderLog
    infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog["isMsg"] == 1) 
                        & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
    del orderLog
    infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
    infoData['Price'] = infoData['Price'].astype('int64')*100
    infoData['OrderQty'] = infoData['absOrderSize']
    infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
    infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
    infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
    statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
    statusInfo.columns = ['order', 'statusLs']
    tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['order', 'TradePriceLs']
    tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['order', 'TradeQtyLs']
    infoData = infoData[infoData['updateType'] == 0]
    infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
    infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])['colo_account'].transform('count')
    display(infoData[infoData['brokerNum'] >= 2].groupby(['colo', 'accCode'])['date'].count())
    display('%.2f%%'%(infoData[infoData['brokerNum'] >= 2].shape[0] / infoData.shape[0]*100))
    display(infoData[infoData['brokerNum'] >= 2].shape[0])
    display(infoData.shape[0])
    infoData = infoData[infoData['brokerNum'] == 1]
    infoData = infoData[['date', 'secid', 'vai', 'ars', "mrstaat", "mrstauc", 'exchange', 'group', 'Price', 'OrderQty', 
                         'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum', 'order', 'colo', 'accCode', 
                         'clockAtArrival', 'orderDirection', 'absOrderSize']]   

    print('finish our orders')

    # 3. find the position of our orders
    checkLog = msgDataSZ[msgDataSZ['updateType'] == 0]
    checkLog = pd.merge(checkLog, infoData.drop_duplicates(subset=['secid', 'ApplSeqNum'])[['secid', 'ApplSeqNum', 'ars']], 
                    on=['secid', 'ApplSeqNum'], how='left')
    checkLog['ApplSeqNum'] = np.where(~checkLog['ars'].isnull(), checkLog['ApplSeqNum'], checkLog['ars'])
    checkLog['start_time'] = np.where(~checkLog['ars'].isnull(), checkLog['clockAtArrival'], checkLog['ars'])
    checkLog.drop(["ars"],axis=1,inplace=True)
    checkLog['ApplSeqNum'] = checkLog.groupby(['secid'])['ApplSeqNum'].ffill()
    checkLog['start_time'] = checkLog.groupby(['secid'])['start_time'].ffill()
    checkLog['end_time'] = checkLog['start_time'] + 100*1e3
    checkLog = checkLog[(~checkLog['ApplSeqNum'].isnull()) & (checkLog['clockAtArrival'] > checkLog['start_time']) & 
                       (checkLog['clockAtArrival'] < checkLog['end_time'])]
    msgDataSZ = msgDataSZ[msgDataSZ['StockSeqNum'].isin(checkLog['StockSeqNum'].values)]
    print('finish get the interval')

    statusInfo = msgDataSZ.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    statusInfo.columns = ['StockSeqNum', 'statusLs']
    tradePriceInfo = msgDataSZ.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
    tradeQtyInfo = msgDataSZ.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
    del msgDataSZ
    checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')  
    del statusInfo
    del tradePriceInfo
    del tradeQtyInfo
    checkLog = checkLog.rename(columns={'clockAtArrival':'caa_orderLog'})

    try:
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], validate='many_to_one')
    except:
        print('There are orders with same pattern but different vai, same ApplSeqNum')
        display([infoData[infoData.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]])
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])
    del infoData
    checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
    checkLog = checkLog[~checkLog['accCode'].isnull()]
    checkLog = checkLog.reset_index(drop=True)
    if original_data[original_data.duplicated(['date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)].shape[0] == 0:
        original_data = pd.merge(original_data, checkLog[['caa_orderLog', 'start_time', 'date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs']], validate='one_to_one',
                                on=['date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], how='left')
    else:
        original_data = pd.merge(original_data, checkLog[['caa_orderLog', 'start_time', 'date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs']], validate='many_to_one',
                                on=['date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], how='left')    

    print(datetime.datetime.now() - startTm)



    def DB(host, db_name, user, passwd):
        auth_db = db_name if user not in ('admin', 'root') else 'admin'
        url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
        client = pymongo.MongoClient(url, maxPoolSize=None)
        db = client[db_name]
        return db

    def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
        collection = db[name]
        # Build projection
        prj = {'_id': 0}
        if col is not None:
            if return_sdi:
                col = ['skey', 'date', 'interval'] + col
            for col_name in col:
                prj[col_name] = 1

        # Build query
        query = {}
        if skey is not None:
            query['skey'] = {'$in': skey}
        if index_id is not None:
            query['index_id'] = {'$in': index_id}
        if interval is not None:
            query['interval'] = {'$in': interval}
        if start_date is not None:
            if end_date is not None:
                query['date'] = {'$gte': start_date, '$lte': end_date}
            else:
                query['date'] = {'$gte': start_date}
        elif end_date is not None:
            query['date'] = {'$lte': end_date}

        # Load data
        cur = collection.find(query, prj)
        df = pd.DataFrame.from_records(cur)
        if df.empty:
            df = pd.DataFrame()
        else:
            df = df.sort_values(by=['date', 'index_id', 'skey'])
        return df    

    def read_beta_daily(db, name, start_date=None, end_date=None, skey=None, interval=None, col=None, return_sdi=True): 
        collection = db[name] 
        # Build projection 
        prj = {'_id': 0} 
        if col is not None: 
            if return_sdi: 
                col = ['skey', 'date'] + col 
            for col_name in col: 
                prj[col_name] = 1 

        # Build query 
        query = {} 
        if skey is not None: 
            query['skey'] = {'$in': skey} 
        if interval is not None: 
            query['interval'] = {'$in': interval} 
        if start_date is not None: 
            if end_date is not None: 
                query['date'] = {'$gte': start_date, '$lte': end_date} 
            else: 
                query['date'] = {'$gte': start_date} 
        elif end_date is not None: 
            query['date'] = {'$lte': end_date} 

        # Load data 
        cur = collection.find(query, prj) 
        df = pd.DataFrame.from_records(cur) 
        if df.empty: 
            df = pd.DataFrame() 
        else: 
            df = df.sort_values(by=['date','skey']) 
        return df  


    database_name = 'com_md_eq_cn'
    user = "zhenyuy"
    password = "bnONBrzSMGoE"

    pd.set_option('max_columns', 200)
    db1 = DB("192.168.10.178", database_name, user, password)


    import os
    import glob
    import datetime
    import numpy as np
    import pandas as pd

    pd.set_option('max_rows', 100)
    pd.set_option('max_columns', 100)

    perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

    print('start generate speed')
    startTm = datetime.datetime.now()

    readPath = '/mnt/equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

    date = dateLs[0]
    assert(date == dd)

    index = read_memb_daily(db1, 'index_memb', int(startDate), int(endDate), index_id=[1000300, 1000905, 1000852, 1000985])
    index = index[['index_id', 'skey', 'date']].reset_index(drop=True)
    index = index.rename(columns={'index_id':'indexCat', 'skey':'secid'})
    index.loc[index['indexCat'] == 1000985, 'indexCat'] = 1000852
    index['ID'] = index['secid']

    d = read_beta_daily(db1, 'mktbeta', int(startDate), int(endDate))
    d1 = d[['skey', 'beta_60d_IF', 'date']]
    d1 = d1.rename(columns={'beta_60d_IF':"beta_60"})
    d1['indexCat'] = 1000300
    d2 = d[['skey', 'beta_60d_IC', 'date']]
    d2 = d2.rename(columns={'beta_60d_IC':"beta_60"})
    d2['indexCat'] = 1000905
    d3 = d[['skey', 'beta_60d_CSI1000', 'date']]
    d3 = d3.rename(columns={'beta_60d_CSI1000':"beta_60"})
    d3['indexCat'] = 1000852
    betaData = pd.concat([d1, d2, d3]).reset_index(drop=True)
    betaData['date'] = betaData['date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d').date())
    betaData = betaData.rename(columns={'skey':'secid'})


    dateDate = datetime.datetime.strptime(date, '%Y%m%d').date()
    dateBetaData = betaData[betaData['date'] == dateDate]

    readPath = '/mnt/equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%date))
    for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        orderLog[col] = orderLog[col].fillna(0)
        orderLog[col] = orderLog[col].astype('int64') 
    orderLog = orderLog.rename(columns={'mdClockAtArrival': 'caamd'})
    
    display('There are accounts with duplicated ticks:')
    display(orderLog[orderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)]\
    .groupby(['date', 'colo', 'accCode'])['ars'].size())
    orderLog = orderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep='first')
    
    display('There are ticks with orderDirection 0')
    display(orderLog[orderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
                'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

    assert(orderLog[orderLog['updateType'] == 0][orderLog[orderLog['updateType'] == 0]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    try:
        assert(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)][orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    except:
        print('There are orders with all things same except sdd')
        print(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)][orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)])
        assert(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)][orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
    try:
        assert(sum(orderLog[(orderLog['updateType'] != 0) & (orderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
    except:
        print('There are orders with same internalId but different orderId other than accCode 8856 case')
        print(orderLog[(orderLog['updateType'] != 0) & (orderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[orderLog[(orderLog['updateType'] != 0) & (orderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

    r2 = orderLog[(orderLog['accCode'] != 8856) & (orderLog['orderDirection'] != 0)]
    r1 = orderLog[(orderLog['accCode'] == 8856) & (orderLog['orderDirection'] != 0)]
    r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize']).grouper.group_info[0]
    r1 = r1.sort_values(by=['test', 'clockAtArrival'])
    r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
    r1['vai'] = r1.groupby('test')['vai'].ffill()
    r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    r2 = r2.sort_values(by=['test', 'clockAtArrival'])
    r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
    r2['vai'] = r2.groupby('test')['vai'].ffill()
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    try:
        assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    orderLog = pd.concat([r1, r2])
    del r1
    del r2  

    orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
    orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize']).grouper.group_info[0]
    orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')
    orderLog = orderLog[orderLog['firstUpdateType'] == 0]

    orderLog['insertNum'] = np.where(orderLog['updateType'] == 0, 1, 0)
    orderLog['insertNum'] = orderLog.groupby(['order'])['insertNum'].transform('sum')
    orderLog = orderLog[orderLog['insertNum'] == 1]

    orderLog['innerSeq'] = orderLog.index.values
    targetStockLs = orderLog['secid'].unique()
    orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')
    orderLog['firstClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')

    assert(orderLog[orderLog['firstUpdateType'] != 0].shape[0] == 0)
    indexCatData = index[index['date'] == int(date)]
    orderLog = pd.merge(orderLog, indexCatData[['secid', 'indexCat']], how='left', on=['secid'], validate='many_to_one')
    orderLog = pd.merge(orderLog, dateBetaData[['secid', 'indexCat', 'beta_60']], how='left', on=['secid', 'indexCat'], validate='many_to_one')

    readPath = '/mnt/Kevin_zhenyu/rawData/logs_%s_***'%date
    mdDataSHPath = glob.glob(os.path.join(readPath, 'mdLog_SH***.csv'))[-1]
    mdDataSH = pd.read_csv(mdDataSHPath)
    mdDataSH['ID'] = mdDataSH['StockID'] + 1000000
    mdDataSH['time'] = mdDataSH.time.str.slice(0, 2) + mdDataSH.time.str.slice(3, 5) + mdDataSH.time.str.slice(6, 8) + '000'
    mdDataSH['time'] = mdDataSH['time'].astype('int64')
    mdDataSH['time'] = mdDataSH.groupby(['ID'])['time'].cummax()
    mdDataSH['max_cum_volume'] = mdDataSH.groupby(['StockID'])['cum_volume'].cummax()
    indexData = mdDataSH[mdDataSH['StockID'].isin([300, 852, 905])][['ID', 'sequenceNo', 'close']].reset_index(drop=True)
    mdDataSH = mdDataSH[mdDataSH['StockID'] >= 600000]
    mdDataSH = mdDataSH[(mdDataSH['cum_volume'] > 0) & (mdDataSH['time'] >= 93000000) &\
                        (mdDataSH['cum_volume'] == mdDataSH['max_cum_volume'])]
    mdDataSH = mdDataSH[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]

    mdDataSZPath = glob.glob(os.path.join(readPath, 'mdLog_SZ***.csv'))[-1]
    mdDataSZ = pd.read_csv(mdDataSZPath)
    mdDataSZ['ID'] = mdDataSZ['StockID'] + 2000000
    mdDataSZ['time'] = mdDataSZ.time.str.slice(0, 2) + mdDataSZ.time.str.slice(3, 5) + mdDataSZ.time.str.slice(6, 8) + '000'
    mdDataSZ['time'] = mdDataSZ['time'].astype('int64')
    mdDataSZ['time'] = mdDataSZ.groupby(['ID'])['time'].cummax()
    mdDataSZ['max_cum_volume'] = mdDataSZ.groupby(['StockID'])['cum_volume'].cummax()
    mdDataSZ = mdDataSZ[(mdDataSZ['cum_volume'] > 0) & (mdDataSZ['time'] >= 93000000) &\
                        (mdDataSZ['cum_volume'] == mdDataSZ['max_cum_volume'])]
    mdDataSZ = mdDataSZ[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]

    mdData = pd.concat([mdDataSH, mdDataSZ, indexData]).reset_index(drop=True)
    mdData = mdData.sort_values(by=['sequenceNo']).reset_index(drop=True)

    addData = pd.DataFrame({'indexCat': [1000300, 1000905, 1000852], 'ID': [1000300, 1000905, 1000852], 
                            'secid':[1000300, 1000905, 1000852]})
    indexCatData = pd.concat([indexCatData, addData], sort=False).reset_index(drop=True)
    mdData = pd.merge(mdData, indexCatData, how='left', on=['ID'], validate='many_to_one')
    mdData = mdData[~mdData['indexCat'].isnull()].reset_index(drop=True)
    mdData['indexClose'] = np.where(mdData['ID'].isin([1000300, 1000852, 1000905]), mdData['close'], np.nan)
    mdData['indexClose'] = mdData.groupby(['indexCat'])['indexClose'].ffill()
    mdData = mdData[~mdData['ID'].isin([1000300, 1000852, 1000905])].reset_index(drop=True)

    mdData = mdData.sort_values(by=['ID', 'sequenceNo']).reset_index(drop=True)
    mdData['safeBid1p'] = np.where(mdData['bid1p'] == 0, mdData['ask1p'], mdData['bid1p'])
    mdData['safeAsk1p'] = np.where(mdData['ask1p'] == 0, mdData['bid1p'], mdData['ask1p'])
    mdData['adjMid'] = (mdData['safeBid1p']*mdData['ask1q'] + mdData['safeAsk1p']*mdData['bid1q'])/(mdData['bid1q'] + mdData['ask1q'])

    mdData['session'] = np.where(mdData['time'] >= 130000000, 1, 0)
    def findTmValue(clockLs, tm, method='L', buffer=0):
        maxIx = len(clockLs)
        orignIx = np.arange(maxIx)
        if method == 'F':
            ix = np.searchsorted(clockLs, clockLs+(tm-buffer))
            ## if target future index is next tick, mask
            mask = (orignIx == (ix - 1))|(orignIx == ix)|(ix == maxIx)
        elif method == 'L':
            ## if target future index is last tick, mask
            ix = np.searchsorted(clockLs, clockLs-(tm-buffer))
            ix = ix - 1
            ix[ix<0] = 0
            ## !!!ATTENTION: model3 change
            mask = (orignIx == ix) | ((clockLs-(tm-buffer)).values < clockLs.values[0])
        ix[mask] = -1
        return ix

    mdData = mdData.reset_index(drop=True)
    groupAllData = mdData.groupby(['ID', 'session'])
    mdData['sessionStartCLA'] = groupAllData['clockAtArrival'].transform('min')
    mdData['relativeClock'] = mdData['clockAtArrival'] - mdData['sessionStartCLA']
    mdData['trainFlag'] = np.where(mdData['relativeClock'] > 179.5*1e6, 1, 0)
    mdData['index'] = mdData.index.values
    mdData['sessionStartIx'] = groupAllData['index'].transform('min')
    for tm in [30, 90, 300]:
        tmCol = 'F{}s_ix'.format(tm)
        mdData[tmCol] = groupAllData['relativeClock'].transform(lambda x: findTmValue(x, tm*1e6, 'F', 5*1e5)).astype(int)
    nearLimit = ((mdData.ask5q.values == 0) | (mdData.bid5q.values == 0))

    for tm in [30, 90, 300]:
        tmIx = mdData['F{}s_ix'.format(tm)].values + mdData['sessionStartIx'].values
        adjMid_tm = mdData['adjMid'].values[tmIx]
        adjMid_tm[mdData['F{}s_ix'.format(tm)].values == -1] = np.nan
        mdData['adjMid_F{}s'.format(tm)] = adjMid_tm

    for tm in [30, 90, 300]:
        tmIx = mdData['F{}s_ix'.format(tm)].values + mdData['sessionStartIx'].values
        adjMid_tm = mdData['indexClose'].values[tmIx]
        adjMid_tm[mdData['F{}s_ix'.format(tm)].values == -1] = np.nan
        mdData['indexClose_F{}s'.format(tm)] = adjMid_tm

    mdData = mdData[mdData['ID'].isin(targetStockLs)]
    mdStartPos = mdData.drop_duplicates(subset=['ID', 'cum_volume'], keep='last')
    mdStartPos = mdStartPos[['ID', 'cum_volume', 'clockAtArrival']].reset_index(drop=True)
    mdStartPos.columns = ['secid', 'vai', 'mdStartClock']
    mdStartPos['isOrder'] = 0
    tradeStartPos = orderLog[orderLog['updateType'] == 0][['secid', 'vai', 'order']].reset_index(drop=True)
    tradeStartPos['isOrder'] = 1
    tradeStartPos = pd.concat([mdStartPos, tradeStartPos], sort=False)
    tradeStartPos = tradeStartPos.sort_values(by=['secid', 'vai', 'isOrder'])
    tradeStartPos['mdStartClock'] = tradeStartPos.groupby(['secid'])['mdStartClock'].ffill()
    tradeStartPos['mdStartClock'] = tradeStartPos.groupby(['secid'])['mdStartClock'].backfill()
    tradeStartPos = tradeStartPos[tradeStartPos['isOrder'] == 1][['secid', 'vai', 'order', 'mdStartClock']]

    orderLog = pd.merge(orderLog, tradeStartPos[['order', 'mdStartClock']], how='left', on=['order'], validate='many_to_one')
    orderLog['mdClockAtArrival'] = orderLog['clockAtArrival'] - orderLog['caamd'] + orderLog['mdStartClock']

    tradeData = orderLog[['secid', 'mdClockAtArrival', 'innerSeq']].reset_index(drop=True)
    tradeData.columns = ['ID', 'clockAtArrival', 'innerSeq']
    tradeData['isOrder'] = 1

    mdData = pd.concat([mdData, tradeData], sort=False)
    mdData = mdData.sort_values(by=['ID', 'clockAtArrival', 'isOrder', 'innerSeq']).reset_index(drop=True)
    for col in ['indexClose', 'adjMid_F30s', 'indexClose_F30s', 'adjMid_F90s', 'indexClose_F90s',
                'adjMid_F300s', 'indexClose_F300s']:
        mdData[col] = mdData.groupby(['ID'])[col].backfill()
        mdData[col] = mdData.groupby(['ID'])[col].ffill()

    tradeData = mdData[mdData['isOrder'] == 1][['ID', 'innerSeq', 'adjMid_F30s', 'adjMid_F90s', 'adjMid_F300s',
                                                'indexClose', 'indexClose_F30s', 'indexClose_F90s', 'indexClose_F300s']].reset_index(drop=True)
    tradeData = tradeData.rename(columns={'ID': 'secid'})
    orderLog = pd.merge(orderLog, tradeData, how='left', on=['secid', 'innerSeq'], validate='one_to_one')

    if original_data[original_data.duplicated(['date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)].shape[0] == 0:
        original_data = pd.merge(original_data, orderLog[['beta_60', 'adjMid_F30s', 'adjMid_F90s', 'adjMid_F300s', 'indexClose', \
        'indexClose_F30s', 'indexClose_F90s', 'indexClose_F300s', 'date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
        'orderDirection', 'absOrderSize']], validate='one_to_one', on=['date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], how='left')
    else:
        original_data = pd.merge(original_data, orderLog[['beta_60', 'adjMid_F30s', 'adjMid_F90s', 'adjMid_F300s', 'indexClose', \
        'indexClose_F30s', 'indexClose_F90s', 'indexClose_F300s', 'date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
        'orderDirection', 'absOrderSize']], validate='many_to_one', on=['date', 'secid', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], how='left')  
    del orderLog
    original_data.to_pickle('/mnt/orderLog/data/' + startDate + '.pkl')
    del original_data
    print(datetime.datetime.now() - startTm)

In [ ]:
for d in ['20201116', '20201117', '20201118', '20201119', '20201120',\
          '20201123', '20201124', '20201125', '20201126', '20201127']:
    download_data(d)

start processing raw Data


/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (52,60) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0:16:18.357603
start tickToMBD


'There are accounts with duplicated ticks:'

Series([], Name: ars, dtype: int64)

...

Series([], Name: ars, dtype: int64)

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
21835,20201120,zs_64_01,6480,2002588,-1,1,40612.0,0,0,-1.0,8.495400e+04
25023,20201120,zs_64_01,6480,2002548,-1,1,50434.0,0,0,-1.0,1.119890e+05
26113,20201120,zs_64_01,6480,2002732,-1,1,52199.0,0,0,-1.0,1.270590e+05
170473,20201120,zs_54_01,5470,2002571,-1,7,49023.0,0,0,-1.0,8.689071e+08
228163,20201120,zs_52_06,5269,2000610,-1,1,49400.0,0,0,-1.0,1.086587e+06
229621,20201120,zs_52_06,5269,2002735,-1,1,52599.0,0,0,-1.0,1.288279e+06
294884,20201120,zt_88_03,8833,2002967,-1,1,35559.0,0,0,-1.0,2.100035e+07
313955,20201120,zs_52_09,5291,2300459,-1,1,37825.0,0,0,-1.0,6.181830e+05
552476,20201120,zs_66_01,6634,2300218,-1,1,34391.0,0,0,-1.0,1.812801e+10
557909,20201120,zs_66_01,6634,2000558,-1,1,46910.0,0,0,-1.0,1.812806e+10


/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:532: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0:15:38.080429
start processing raw Data
